# import nessecary package and choose processeing files

In [1]:
#import important modules and packages
from pypl2 import pl2_ad, pl2_spikes, pl2_events, pl2_info
import sys
import os
import numpy as np
import time
import scipy.io as sio

#choose files,
#get the filename of the pl2 files
def choose_file():
    try:
        import Tkinter, tkFileDialog
    except ImportError:
        #print "Tkinter not installed."
        print "Tkinter not installed."
        exit()
    
    #Suppress the Tkinter root window
    tkroot = Tkinter.Tk()
    tkroot.withdraw()
    
    return str(tkFileDialog.askopenfilename())
    
if __name__ == "__main__":
    
    #If no file is passed at the command line, or if the file
    #passed can not be found, open a file chooser window.
    if len(sys.argv) < 2:
        filename = os.path.abspath(choose_file())
    else:
        filename = os.path.abspath(sys.argv[1])
        if not os.path.isfile(filename):
            filename = choose_file()
            

print "The files being process is "+filename

The files being process is E:/Panda/PandaData/DatawithTimestamp/102/20190108_102.pl2


In [2]:
#Get file info and print out interesting information, for chooseing the channel in WB
spkinfo, evtinfo, adinfo = pl2_info(filename)
print "\nContinuous A/D Channel Info from pl2_info()"
print "\nlength of recording is: "+str(adinfo[0].n/40000) + " seconds"  #40kHz sampling rate for wb data
print "\n# Channel Name\tCount"
print "- ------------\t-----"
for n in range(len(adinfo)):
    print "%s %-10s\t%s" % (adinfo[n].channel, adinfo[n].name, adinfo[n].n)


Continuous A/D Channel Info from pl2_info()

length of recording is: 1079 seconds

# Channel Name	Count
- ------------	-----
1 WB01      	43199971
2 WB02      	43199971
3 WB03      	43199971
4 WB04      	43199971
5 WB05      	43199971
6 WB06      	43199971
7 WB07      	43199971
8 WB08      	43199971
9 WB09      	43199971
10 WB10      	43199971
11 WB11      	43199971
12 WB12      	43199971
13 WB13      	43199971
14 WB14      	43199971
15 WB15      	43199971
16 WB16      	43199971
1 SPKC01    	43199971
2 SPKC02    	43199971
3 SPKC03    	43199971
4 SPKC04    	43199971
5 SPKC05    	43199971
6 SPKC06    	43199971
7 SPKC07    	43199971
8 SPKC08    	43199971
9 SPKC09    	43199971
10 SPKC10    	43199971
11 SPKC11    	43199971
12 SPKC12    	43199971
13 SPKC13    	43199971
14 SPKC14    	43199971
15 SPKC15    	43199971
16 SPKC16    	43199971
1 FP01      	1080000
2 FP02      	1080000
3 FP03      	1080000
4 FP04      	1080000
5 FP05      	1080000
6 FP06      	1080000
7 FP07      	1080000
8 FP08   

In [3]:
#get the channel in WB, may reject those broken channel here
wbChannel = list()
for i in range(len(adinfo)):
    name = adinfo[i].name
    if 'WB' in name:
        wbChannel.append(name)
wbChannel

['WB01',
 'WB02',
 'WB03',
 'WB04',
 'WB05',
 'WB06',
 'WB07',
 'WB08',
 'WB09',
 'WB10',
 'WB11',
 'WB12',
 'WB13',
 'WB14',
 'WB15',
 'WB16']

In [4]:
#get the filename for np array save in other place
base=os.path.basename(filename)
savefilename = os.path.splitext(base)[0]
savepath ='F:/shift/recording/dataset'+savefilename
#savepath ='C:/Users/309i7/Downloads/mountainsort_examples/jupyter_examples/001_ms4_jupyter_example/nparrayfrompl2/'
print "The new files will be saved in this path: " + savepath
print "The new np array will be saved with the name: "+ savefilename

if not os.path.exists(savepath):
    os.mkdir(savepath)

The new files will be saved in this path: F:/shift/recording/dataset20190108_102
The new np array will be saved with the name: 20190108_102


In [7]:
#save using channel one by one in binary file,    
#write each channel (WB data into .dat file, ie. raw binary file for mountainsort)
t0 = time.time()

for i in wbChannel:
    #set the folder to save dataset
    filepath = savepath+'/'+i
    if not os.path.exists(filepath):
        os.mkdir(filepath)
        
        
    #directory struce:
    #-shift/recording/datasetxxx/WBxx/xxxWBxxch.dat (xxx:date+rat no.; xx:channel)
    f = open(filepath +'/'+savefilename+i+"ch.dat", "wb")
    ad = pl2_ad(filename,i)
    
    #load each channel data
    wb = np.array(ad.ad)    
    #write to .dat file
    wb.tofile(f)
    
    
    print i
    f.close()

t1 = time.time() - t0
print t1

WB01
WB02
WB03
WB04
WB05
WB06
WB07
WB08
WB09
WB10
WB11
WB12
WB13
WB14
WB15
WB16
279.082000017


In [ ]:
#read from
data1 = np.fromfile('data2.dat', dtype="float64")

In [ ]:
#copy this to the terminal for .dat to .mda conversion
#dimesion: (-1, channels), -1 indicates auto-calculation
#channel: (zeros-index based, include all channels you want to sort)
#dtype & dtype_out:  float64

#ml-run-process ephys.convert_array \--inputs \input:/mnt/e/Panda/PandaCode/PyPL2/data.dat --outputs \output:/mn
#t/e/Panda/PandaCode/PyPL2/data.mda --parameters \format:dat format_out:mda dtype:float64 dtype_out:float64 channels:0,1,2,3 dimensions:-1,4

# then go to the linux system for mountain sorting! 